In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf

import time

import cv2
import os

# Main slim library
from tensorflow.contrib import slim

## colab에서 실행하세요~

In [ ]:
!git clone https://github.com/hukim1112/DLCV_CLASS.git
os.chdir('/content/DLCV_CLASS')
from datasets import dataset_utils

## fine tunning할 데이터 다운로드

In [ ]:
_DATA_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
data_dir = '/content/sample_data'
dataset_utils.download_and_uncompress_tarball(_DATA_URL, data_dir)

## pretrained model 다운로드

In [ ]:
_CHECKPOINT_URL = "http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz"
checkpoints_dir = '/content/sample_data'

if not tf.gfile.Exists(checkpoints_dir):
    tf.gfile.MakeDirs(checkpoints_dir)

dataset_utils.download_and_uncompress_tarball(_CHECKPOINT_URL, checkpoints_dir)

## declare few paths 

In [ ]:
dataset_path = "/content/sample_data"
checkpoint_path = "/content/sample_data"
train_dir = '../inception_finetuned/'

## input pipeline (이 부분은 넘기셔도 됩니다.)

In [ ]:
def _get_filenames_and_classes(dataset_dir):
      flower_root = os.path.join(dataset_dir, 'flower_photos')
      directories = []
      class_names = []
      for dir_name in os.listdir(flower_root):
        path = os.path.join(flower_root, dir_name)
        if os.path.isdir(path):
          directories.append(path)
          class_names.append(dir_name)

      photo_filenames = []
      for directory in directories:
        for filename in os.listdir(directory):
          path = os.path.join(directory, filename)
          photo_filenames.append(path)

      return photo_filenames, sorted(class_names)

In [ ]:
filepaths, class_names = _get_filenames_and_classes(dataset_path)
class_names_to_ids = dict(zip(class_names, range(len(class_names))))
ids_to_class_name = dict(zip(range(len(class_names)), class_names))

In [ ]:
from random import shuffle
shuffle(filepaths)

In [ ]:
train_data = filepaths[:3000]
validate_data = filepaths[3000:]

In [ ]:
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    images_decoded = tf.image.decode_jpeg(image_string, channels = 3)
    raw_images = tf.image.resize_images(images_decoded, [224, 224])
    images = inception_preprocessing.preprocess_image(images_decoded, height = 224, width = 224, is_training=True)
    
    return images, raw_images, label

### without dataset pipeline

<p align="center">
    <img src="https://github.com/hukim1112/DLCV_CLASS/blob/master/datasets_without_pipelining.png?raw=true" width=600></br>
</p>

### with dataset pipeline

<p align="center">
    <img src="https://github.com/hukim1112/DLCV_CLASS/blob/master/datasets_with_pipelining.png?raw=true" width=600></br>
</p>

In [ ]:
from preprocessing import inception_preprocessing
import tensorflow as tf

from tensorflow.contrib import slim


def load_batch(filepaths, num_images, batch_size=32):

    dataset_filepath = tf.data.Dataset.from_tensor_slices(tf.cast(filepaths, tf.string))
    dataset_class = tf.data.Dataset.from_tensor_slices(
        [class_names_to_ids[os.path.basename(os.path.dirname(filepath))] for filepath in filepaths])
    
    dataset = tf.data.Dataset.zip((dataset_filepath, dataset_class))
    dataset = dataset.shuffle(num_images)
    dataset = dataset.repeat()
    dataset = dataset.map(_parse_function, num_parallel_calls=4)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(2)
    iterator = dataset.make_one_shot_iterator()
    images, raw_images, labels = iterator.get_next()


    return images, raw_images, labels

In [ ]:
images, raw_images, labels = load_batch(filepaths, num_images = len(filepaths), batch_size=32)
# with tf.Session() as sess:
#     for i in range(1):
#         a = sess.run(images)

#print(a.shape)
images.shape

# TensorFlow는 graph 예시 (실행 X)

In [1]:
# 실행 X
def my_cnn(images, num_classes, is_training):  # is_training is not used...
    with slim.arg_scope([slim.max_pool2d], kernel_size=[3, 3], stride=2):
        net = slim.conv2d(images, 64, [5, 5])
        net = slim.max_pool2d(net)
        net = slim.conv2d(net, 64, [5, 5])
        net = slim.max_pool2d(net)
        net = slim.flatten(net)
        net = slim.fully_connected(net, 192)
        net = slim.fully_connected(net, num_classes, activation_fn=None)       
        return net

In [3]:
images, _, labels = load_batch(train_data, num_images = len(filepaths))

scores = my_cnn(images, 5, is_training=True)


NameError: name 'load_batch' is not defined

In [ ]:
# 실행 X
class AE():
    def __init__(self):
        self.graph = tf.Graph()
        self.batch_size = 128
        with self.graph.as_default():
            with tf.device('/cpu:0'):
                self.images, labels = get_data.data_pipeline('mnist', 128, 'train')
            self.visualizer = Visualizer(exp_name='AE_mnist', row=8, col=8)
            self.generated_images = self._build_graph(self.images, 5, reuse=tf.AUTO_REUSE, training=True)
            self.loss = self._loss_function(self.images, self.generated_images)
            self.solver = tf.train.AdamOptimizer(learning_rate=0.0001) \
                           .minimize(self.loss)
            initializer = tf.global_variables_initializer()
            self.sess = tf.Session()
            self.sess.run(initializer)
    def train(self, iteration):
        for i in range(iteration+1):
            loss, _, np_real_images, np_generated_images = self.sess.run(
                    [self.loss, self.solver, self.images, self.generated_images])
            if i % 10 == 0:
                print("iterator {} : loss {} ".format(i, loss))
            if i % 1000 == 0:
                self.visualize(np_real_images, np_generated_images, i)
            
    def visualize(self, images, generated_images, i):
        visual_imgs = np.concatenate( (images[:8*4], generated_images[:8*4]), axis = 0 )
        visual_imgs = 255*(visual_imgs/2 + 0.5)
        visual_imgs = visual_imgs.astype(int)
        self.visualizer.draw_imgs(visual_imgs)
        self.visualizer.save_fig(name="AE_mnist_iter_{}".format(str(i)))
        
    def _build_graph(self, input, dim_code, reuse=tf.AUTO_REUSE, training=False):
        regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)
        net = layers.conv2d(input, 64, (3,3), 1, weights_regularizer=regularizer)
        net = layers.conv2d(net, 32, (3,3), 2, weights_regularizer=regularizer)
        net = layers.conv2d(net, 32, (3,3), 2, weights_regularizer=regularizer)
        net = layers.flatten(net)
        net = layers.fully_connected(net, 128, weights_regularizer=regularizer)
        latent_var = layers.fully_connected(net, dim_code, weights_regularizer=regularizer)
        
        net = layers.fully_connected(latent_var, 128, weights_regularizer=regularizer)
        net = layers.fully_connected(latent_var, 7*7*32, weights_regularizer=regularizer)
        net = tf.reshape(shape=[-1, 7, 7, 32], tensor=net)
        net = layers.conv2d_transpose(net, 32, (3,3), 1, weights_regularizer=regularizer)
        net = layers.conv2d_transpose(net, 32, (3,3), 2, weights_regularizer=regularizer)
        net = layers.conv2d_transpose(net, 64, (3,3), 2, weights_regularizer=regularizer)
        net = layers.conv2d_transpose(net, 1, (3,3), 1, activation_fn=tf.nn.tanh, weights_regularizer=regularizer)
        return net
    def _loss_function(self, _real_images, _generated_images):
        recon_loss = tf.reduce_mean(tf.square(_real_images - _generated_images))
        return recon_loss

# Fine-tune the model on a different set of labels.

In [ ]:
# Note that this may take several minutes.

import os

from datasets import flowers
from nets import inception
from preprocessing import inception_preprocessing

from tensorflow.contrib import slim
image_size = inception.inception_v1.default_image_size


def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    checkpoint_exclude_scopes=["InceptionV1/Logits", "InceptionV1/AuxLogits"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)

    return slim.assign_from_checkpoint_fn(
      os.path.join(checkpoint_path, 'inception_v1.ckpt'),
      variables_to_restore)



with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
    images, _, labels = load_batch(train_data, num_images = len(filepaths))
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes= len(class_names), is_training=True)
        
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, num_classes = len(class_names))
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # Run the training:
    final_loss = slim.learning.train(
        train_op,
        logdir=train_dir,
        init_fn=get_init_fn(),
        number_of_steps=100)
        
  
print('Finished training. Last batch loss %f' % final_loss)

# Evaluation

In [ ]:
import numpy as np
import tensorflow as tf
from datasets import flowers
from nets import inception

from tensorflow.contrib import slim

image_size = inception.inception_v1.default_image_size
batch_size = 10

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    

    images, images_raw, labels = load_batch(validate_data, num_images = len(filepaths))
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
        logits, _ = inception.inception_v1(images, num_classes= len(class_names), is_training=True)
    predictions = tf.argmax(logits, 1)
    probabilities = tf.nn.softmax(logits)
    
    checkpoint_path = tf.train.latest_checkpoint(train_dir)
    init_fn = slim.assign_from_checkpoint_fn(
      checkpoint_path,
      slim.get_variables_to_restore())
    
    with tf.Session() as sess:
        with slim.queues.QueueRunners(sess):
            sess.run(tf.initialize_local_variables())
            init_fn(sess)
            np_probabilities, np_images_raw, np_labels = sess.run([probabilities, images_raw, labels])
    
            for i in range(batch_size): 
                image = np_images_raw[i, :, :, :]
                true_label = np_labels[i]
                predicted_label = np.argmax(np_probabilities[i, :])
                predicted_name = ids_to_class_name[predicted_label]
                true_name = ids_to_class_name[true_label]
                
                plt.figure()
                plt.imshow(image.astype(np.uint8))
                plt.title('Ground Truth: [%s], Prediction [%s]' % (true_name, predicted_name))
                plt.axis('off')
                plt.show()
                
    # Define the metrics:
    names_to_values, names_to_updates = slim.metrics.aggregate_metric_map({
        'eval/Accuracy': slim.metrics.streaming_accuracy(predictions, labels),
        'eval/Recall@5': slim.metrics.streaming_recall_at_k(logits, labels, 5),
    })

    print('Running evaluation Loop...')
    checkpoint_path = tf.train.latest_checkpoint(train_dir)
    metric_values = slim.evaluation.evaluate_once(
        master='',
        checkpoint_path=checkpoint_path,
        logdir=train_dir,
        eval_op=list(names_to_updates.values()),
        final_op=list(names_to_values.values()))    

    names_to_values = dict(zip(names_to_values.keys(), metric_values))
    for name in names_to_values:
        print('%s: %f' % (name, names_to_values[name]))
    
